In [ ]:
import numpy
import xarray as xr
import igutils

dir(igutils)

In [ ]:
grid = xr.open_dataset(
    "icon_grid_0013_R02B04_R.nc"
)
grid

In [ ]:
igutils.check_consistency(grid)

In [ ]:
eov = igutils.find_pentagons_vertices(grid)
print(eov)

In [ ]:
vs = [int(x) for x in grid.vertices_of_vertex[:, eov[0]]]
print(vs[0:-1])
latitudes = [float(l) for l in grid.vertices_of_vertex.vlat[vs[0:-1]]]
print(latitudes)

In [ ]:
# if the longitudes of the are repeated for the two halves of the array then the grid is not an icosahedron but a symmetric grid
print(f"grid.latitude_vertices.values[eov] = {grid.latitude_vertices.values[eov]}")
print(f"grid.longitude_vertices.values[eov] = {grid.longitude_vertices.values[eov]}")

In [ ]:
paths = igutils.pentagons_paths_vertices(grid)

In [ ]:
interesting_path_length = min([ len(x) for x in paths.values() if len(x)>1])
print(f"{interesting_path_length=}")

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
voc = grid.vertices_of_vertex.values

fig = plt.figure(figsize=(50, 40)) # Need to find a way of setting the size right...
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
ax.set_global()
ax.add_feature(cfeature.LAND, zorder=0, edgecolor="black")
pad = ''
v = 0
transformatio = ccrs.Geodetic()
for v1, v2, v3, v4, v5, v6 in grid.vertices_of_vertex.values.T - 1:
    i = np.array([v, v1])
    for vi in [v1, v2, v3, v4, v5]:
        i = np.array([v, vi])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c= 'k' if v6<=0 else "k",
            lw=3 if v6<=0 else 1,
            alpha=1 if v6<=0 else 0.1,
            transform=transformatio,
        )
    if v6 > 0:
        i = np.array([v, v6])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c="k",
            lw=1,
            alpha=0.1,
            transform=transformatio,
        )
    if True: # v6<=0: Put this on to print only the ids of the pentagons (useful if the grid is big)
        plt.text(np.rad2deg(grid.vertices_of_vertex.vlon[v]),
                 np.rad2deg(grid.vertices_of_vertex.vlat[v]),
                 str(v+1),
                 transform=transformatio
                )
    v=v+1
# v = 0
# for v1, v2, v3, v4, v5, v6 in grid.vertices_of_vertex.values.T - 1:
#     i = np.array([v, v1])
#     if v6<=0:
#         print(f"v = {v}")
#         for vi in [v1, v2, v3, v4, v5]:
#             i = np.array([v, vi])
#             plt.plot(
#                 np.rad2deg(grid.vertices_of_vertex.vlon[i]),
#                 np.rad2deg(grid.latitude_vertices.vlat[i]),
#                 c= 'k' if v6<=0 else "k",
#                 lw=3 if v6<=0 else 1,
#                 alpha=1 if v6<=0 else 0.1,
#                 transform=transformatio,
#             )
#         plt.text(np.rad2deg(grid.vertices_of_vertex.vlon[v]),
#                 np.rad2deg(grid.vertices_of_vertex.vlat[v]),
#                 str(v+1),
#                 transform=transformatio
#                 )
#     v=v+1

colors = ['r', 'g', 'y', 'c']
count = 0
for path1 in paths.values():
    # for path1 in thepaths:
        #print(f"[{count}] length of path = {len(path1)} from {path1[0]} [{float(grid.vertices_of_vertex.vlon[path1[0]-1])}, {float(grid.vertices_of_vertex.vlat[path1[0]-1])}] to {path1[-1]} [{float(grid.vertices_of_vertex.vlon[path1[-1]-1])}, {float(grid.vertices_of_vertex.vlat[path1[-1]-1])}]")
        if len(path1) <= interesting_path_length:
            
            for ind in range(len(path1)-1):
                #print(f"from vertex {path1[ind]-1} to {path1[ind+1]-1}")
                i = np.array([path1[ind]-1, path1[ind+1]-1])
                plt.plot(
                    np.rad2deg(grid.longitude_vertices.vlon[i]),
                    np.rad2deg(grid.latitude_vertices.vlat[i]),
                    c=colors[count%len(colors)],
                    lw=4,
                    alpha=1,
                    transform=transformatio,
                )
            count = count+1

plt.show()
plt.close()

In [ ]:

#print(paths.keys())
rhomboids_north, rhomboids_south = igutils.find_rhomboids(eov, grid, paths, interesting_path_length)

In [ ]:
vsequence = igutils.mark_rhomboid(rhomboids_north[3],paths, grid)
len(vsequence)


In [ ]:
voc = grid.vertices_of_vertex.values

fig = plt.figure(figsize=(100, 80)) # Need to find a way of setting the size right...
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
ax.set_global()
ax.add_feature(cfeature.LAND, zorder=0, edgecolor="black")
pad = ''
v = 0
transformatio = ccrs.Geodetic()
for v1, v2, v3, v4, v5, v6 in grid.vertices_of_vertex.values.T - 1:
    i = np.array([v, v1])
    for vi in [v1, v2, v3, v4, v5]:
        i = np.array([v, vi])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c="k",
            lw=1,
            alpha=0.1,
            transform=transformatio,
        )
    if v6 > 0:
        i = np.array([v, v6])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c="k",
            lw=1,
            alpha=0.1,
            transform=transformatio,
        )
    # if v+1 in mark:
    #     plt.scatter([float(np.rad2deg(grid.vertices_of_vertex.vlon[v]))],
    #             [float(np.rad2deg(grid.vertices_of_vertex.vlat[v]))],
    #             c = 'r',
    #             s = 10,
    #             transform=transformatio
    #             )
    v=v+1
plt.scatter(
    [float(np.rad2deg(grid.vlon[x-1])) for x in vsequence],
    [float(np.rad2deg(grid.vlat[x-1])) for x in vsequence],
    c = 'r', s = 15, transform=transformatio
    )

ll = 0
for x in vsequence:
    plt.text(float(np.rad2deg(grid.vlon[x-1])),
            float(np.rad2deg(grid.vlat[x-1])),
            str(ll), transform=transformatio
            )
    ll += 1

plt.show()
plt.close()